In [ ]:
import pyshorteners

s = pyshorteners.Shortener()

import requests
import json
import os
from insta28 import *
import time
import datetime

from PIL import Image
from io import BytesIO

from dotenv import load_dotenv

load_dotenv('.env')
load_dotenv('paramns.env')

In [ ]:
# Obter a data e hora atuais
data_hora_atual = datetime.datetime.now()

# Formatar a data e hora
data_hora_formatada = data_hora_atual.strftime('%Y-%m-%d %H:%M:%S')

timestampNow = int(data_hora_atual.timestamp() * 1000)

In [ ]:
def classImg(
    hydraAPIURL: str
    , hydraAPIHost: str
    , rapidAPIKey: str
    , imgURL: str    
):
    try:
        payload = { "image": imgURL}
        headers = {
            "content-type": "application/json"
            ,"X-RapidAPI-Key": rapidAPIKey
            ,"X-RapidAPI-Host": hydraAPIHost
        }

        try:
            response = requests.post(hydraAPIURL, json=payload, headers=headers)
        except Exception as e:
            return False, e, None

        dictHydra = response.json()
        
        if (dictHydra['statusCode'] == 200):
            return True, dictHydra['body']['image_classification'], dictHydra['body']['object_detection']
        else:
            return False, None, None
        
    except Exception as e:
        if len(dictHydra):
            return False, e, dictHydra
        else:
            return False, e, None

In [ ]:
url = os.getenv('INSTA28URL')

queryHashTag = input("Qual o termo que você quer pesquisar?")
querystring = {"hash_tag":queryHashTag}

headers = {
	"X-RapidAPI-Key": os.getenv('RAPIDAPI'),
	"X-RapidAPI-Host": os.getenv("INSTA28HOST")
}

response = requests.get(url, headers=headers, params=querystring)
dictInsta = response.json()

In [ ]:
inst = Instmages(dictInsta['data'], dictInsta['extensions'], dictInsta['status'])

In [ ]:
hashs = Hashtag(
    inst.data['hashtag']['id']
    , inst.data['hashtag']['name']
    , inst.data['hashtag']['allow_following']
    , inst.data['hashtag']['is_following']
    , inst.data['hashtag']['is_top_media_only']
    , inst.data['hashtag']['profile_pic_url']
    , inst.data['hashtag']['edge_hashtag_to_media']
    , inst.data['hashtag']['edge_hashtag_to_content_advisory']
)

In [ ]:
edges = EdgeHashtagToMedia(
    count = hashs.edge_hashtag_to_media['count']
    ,page_info=hashs.edge_hashtag_to_media['page_info']
    ,edges= hashs.edge_hashtag_to_media['edges']
)

In [ ]:
i = 0
dictResult = {}

dictResult['hashtag'] = queryHashTag
dictResult['datetime'] = data_hora_formatada
dictResult['timestampnow'] = timestampNow

lstPrplNode = []
for edge in edges.edges:
    node = EdgeHashtagToMediaEdge(
        node = edge['node']
    )
    prplNode = PurpleNode(
        comments_disabled = node.node['comments_disabled']
        ,typename = node.node['__typename']
        ,id = node.node['id']
        ,edge_media_to_caption = node.node['edge_media_to_caption']
        ,shortcode = node.node['shortcode']
        ,edge_media_to_comment = node.node['edge_media_to_comment']
        ,taken_at_timestamp = node.node['taken_at_timestamp']
        ,dimensions = node.node['dimensions']
        ,display_url = node.node['display_url']
        ,edge_liked_by = node.node['edge_liked_by']
        ,edge_media_preview_like = node.node['edge_media_preview_like']
        ,owner = node.node['owner']
        ,thumbnail_src = node.node['thumbnail_src']
        ,thumbnail_resources = node.node['thumbnail_resources']
        ,is_video = node.node['is_video']
        ,accessibility_caption = node.node['accessibility_caption']
    )
    
    if prplNode.is_video:
        continue
    
    for caption in prplNode.edge_media_to_caption['edges']:
        node = EdgeMediaToCaptionEdge(
            node = caption['node']
        )
        
        fluffyText = FluffyNode(text = node.node['text'])
        _shortUrl = s.isgd.short(prplNode.display_url)
        
        status, imgClass, objDtct = classImg(
            hydraAPIURL = os.getenv("HYDRAAPIURL")
            , hydraAPIHost = os.getenv("HYDRAAPIHOST")
            , rapidAPIKey = os.getenv("RAPIDAPI")
            , imgURL = prplNode.display_url
        )
        time.sleep(1)
            
        response = requests.get(_shortUrl)

        if response.status_code == 200:
            pasta = f"results/{queryHashTag}"
            if not os.path.exists(pasta):
                os.makedirs(pasta)
                os.makedirs(f"{pasta}/images")
                os.makedirs(f"{pasta}/data")
                
            img = Image.open(BytesIO(response.content))
            img.save(f"{pasta}/images/{prplNode.owner['id']}-{prplNode.id}.jpg")
        else:
            print("Falha ao baixar a imagem. Status code:", response.status_code)
            
        _dctPrpl = {
            "typename": prplNode.typename
            ,"id": prplNode.id
            ,"text": fluffyText.text
            ,"ownerid": prplNode.owner['id']
            ,"localurl": f"{pasta}/images/{prplNode.owner['id']}-{prplNode.id}.jpg"
            ,"originalurl": _shortUrl
            ,"imgClass": imgClass
            ,"objDtct": objDtct
        }
        
        lstPrplNode.append(_dctPrpl)
        
        
    # break
    if i > 4:
        break
    else:
        i += 1
    
dictResult['records'] = lstPrplNode

In [ ]:
with open(f'{pasta}/data/{queryHashTag}-{timestampNow}.json', 'w') as f:
    f.write(json.dumps(dictResult))

In [ ]:
import requests
from PIL import Image
from io import BytesIO

url = "https://scontent-lga3-2.cdninstagram.com/v/t51.2885-15/393463999_844386093735145_8392362185252247702_n.jpg?stp=dst-jpg_e35_s1080x1080&_nc_ht=scontent-lga3-2.cdninstagram.com&_nc_cat=101&_nc_ohc=JDTYr_f63RMAX8ZBWrr&edm=AA0rjkIBAAAA&ccb=7-5&oh=00_AfD_4JLuIkYIAITHGpS9V2hToiNhlF7G7XKgajOgYV14bw&oe=65382FA0&_nc_sid=49ed71"

response = requests.get(url)

if response.status_code == 200:
    img = Image.open(BytesIO(response.content))
    img.save("imagem_salva.jpg")
    print("Imagem salva com sucesso!")
else:
    print("Falha ao baixar a imagem. Status code:", response.status_code)

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Defina o caminho para o diretório do conjunto de dados Caltech-256
dataset_dir = 'caltech-256'

# Listar as classes disponíveis no conjunto de dados
classes = os.listdir(dataset_dir)
num_classes = len(classes)

# Crie listas vazias para armazenar imagens e rótulos
images = []
labels = []

# Carregue imagens e rótulos
for class_name in classes:
    class_dir = os.path.join(dataset_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        if image_path.split(".")[-1] != 'jpg':
            continue
        img = cv2.imread(image_path)
        img = cv2.resize(img, (200, 200))  # Redimensione para o tamanho esperado pelo modelo
        images.append(img)
        labels.append(classes.index(class_name))

# Converta as listas em matrizes numpy
images = np.array(images)
labels = np.array(labels)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Pré-processamento das imagens (normalização, divisão por 255, etc.)
X_train = X_train / 255.0
X_test = X_test / 255.0

# Definir o modelo
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=10)

# Avaliar o modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')



In [ ]:
# Classificar uma imagem de entrada
# Substitua 'caminho_para_sua_imagem.jpg' pelo caminho para a sua imagem
input_image_path = 'imagem_salva.jpg'
img = cv2.imread(input_image_path)
img = cv2.resize(img, (200, 200))  # Redimensione para o tamanho esperado pelo modelo
img = img / 255.0  # Pré-processamento

# Faça a previsão
img = np.expand_dims(img, axis=0)
predictions = model.predict(img)

# Obtenha a classe prevista
predicted_class = np.argmax(predictions)

# Mapeie o índice da classe para o rótulo real
predicted_label = classes[predicted_class]

print(f'Classe prevista para a imagem: {predicted_label}')